# `GiRaFFE_HO` C code library: A-to-B code

## Author: Patrick Nelson

<a id='intro'></a>

## Introduction
$$\label{intro}$$

This writes and documents the C code that `GiRaFFE_HO` uses to compute the magnetic fields from the vector potential. This is a relatively straightforward calculation, but requires care to be taken in the ghost zones. 
These expressions are used to calculate the magnetic fields from the vector potential. 

We will also need to compute $B^i$ everywhere in order to evolve $\tilde{S}_i$. However, $B^i = \epsilon^{ijk} \partial_j A_k$ requires derivatives of $A_i$, so getting $B^i$ will require some finesse. A chief aspect of this will require using lower-order finite differencing in the ghost zones. To that end, we will create header files for each finite differencing order $\leq 10$, as well as upwinded- and downwinded-derivatives at 2nd order. These will let us compute the derivative at the outermost gridpoints.


<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

1. [Step 1](#headers): Creating the header files
    1. [Step 1.a](#outermost): Handling the outermost ghostzones
1. [Step 2](#ccode): C code
    1. [Step 2.a](#a2b): Deciding which header file to use
    1. [Step 2.b](#driver): Looping over all ghostzones

In [1]:
import os
outdir = "GiRaFFE_HO/"

<a id='headers'></a>

## Step 1: Creating the header files \[Back to [top](#toc)\]
$$\label{headers}$$

We start in the usual way - import the modules we need, set `DIM = 3`, and run the function `gho.GiRaFFE_Higher_Order_v2()` to build equations. We will also import the Levi-Civita symbol from `WeylScalars_Cartesian`. 

In [2]:
# The A-to-B driver
from outputC import *
import grid as gri
import indexedexp as ixp
import NRPy_param_funcs as par
import finite_difference as fin

# Set spatial dimension (must be 3 for BSSN)
DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

# Import the GiRaFFE_HO_v2 module and run its function to build BU
import GiRaFFE_HO.GiRaFFE_Higher_Order_v2 as gho
gho.GiRaFFE_Higher_Order_v2()

# Import the Levi-Civita symbol and build the corresponding tensor.
# We already have a handy function to define the Levi-Civita symbol in WeylScalars
import WeylScal4NRPy.WeylScalars_Cartesian as weyl
LeviCivitaDDD = weyl.define_LeviCivitaSymbol_rank3()
LeviCivitaUUU = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            LCijk = LeviCivitaDDD[i][j][k]
            #LeviCivitaDDD[i][j][k] = LCijk * sp.sqrt(gho.gammadet)
            LeviCivitaUUU[i][j][k] = LCijk / sp.sqrt(gho.gammadet)


initialize_Cparam() minor warning: Did nothing; already initialized parameter smallbPoynET::M_PI


We will now create the structures we need: `AD_dD` for $A_{i,j}$ and `BU` for $B^i$. We will also define a quick function `set_BU_to_print()`; this will allow us to quickly rebuild the data structure that we will pass to `fin.FD_outputC()` each time we change `BU`.

In [3]:
AD_dD = ixp.declarerank2("AD_dD","nosym")
BU = ixp.zerorank1() # BU is already registered as a gridfunction, but we need to zero its values and declare it in this scope.
# We can use this function to compactly reset to expressions to print at each FD order.
def set_BU_to_print():
    return [lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
            lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
            lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2])]            

for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]


Now, we can output the header files that will be used to calculate $B^i$ on the interior of the grid and on the non-outermost ghost zones. For each FD order $\leq 10$, we will set the NRPy+ parameter `FD_CENTDERIVS_ORDER` to that order, then run `fin.FD_outputC()`, taking care to use our new function`set_BU_to_print()` to define *what* we are writing to the file.

In [4]:
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 10)
fin.FD_outputC(os.path.join(outdir,"B_from_A_10.h"),set_BU_to_print(),params="outCverbose=False")

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 8)
fin.FD_outputC(os.path.join(outdir,"B_from_A_8.h"),set_BU_to_print(),params="outCverbose=False")

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 6)
fin.FD_outputC(os.path.join(outdir,"B_from_A_6.h"),set_BU_to_print(),params="outCverbose=False")

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 4)
fin.FD_outputC(os.path.join(outdir,"B_from_A_4.h"),set_BU_to_print(),params="outCverbose=False")

par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 2)
fin.FD_outputC(os.path.join(outdir,"B_from_A_2.h"),set_BU_to_print(),params="outCverbose=False")


Wrote to file "GiRaFFE_HO/B_from_A_10.h"
Wrote to file "GiRaFFE_HO/B_from_A_8.h"
Wrote to file "GiRaFFE_HO/B_from_A_6.h"
Wrote to file "GiRaFFE_HO/B_from_A_4.h"
Wrote to file "GiRaFFE_HO/B_from_A_2.h"


<a id='outermost'></a>

### Step 1.a: Handling the outermost ghostzones \[Back to [top](#toc)\]
$$\label{outermost}$$

Finally, we will write the files that will handle the outermost ghost zones. Each of these ghost zones represents one face of a cube. Since there is no data outside the cube, we will use an up- or downwinded derivative so that we use only interior data. For example, on the +x ghost zone, we will use a downwinded derivative in the x-direction. So, we will need to create a total of six files:

In [5]:
# For the outermost points, we'll need a separate file for each face. 
# These will correspond to an upwinded and a downwinded file for each direction.
AD_ddnD = ixp.declarerank2("AD_ddnD","nosym")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 0:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC(os.path.join(outdir,"B_from_A_2x0D.h"),set_BU_to_print(),params="outCverbose=False")

AD_dupD = ixp.declarerank2("AD_dupD","nosym")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 0:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC(os.path.join(outdir,"B_from_A_2x0U.h"),set_BU_to_print(),params="outCverbose=False")

for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 1:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC(os.path.join(outdir,"B_from_A_2x1D.h"),set_BU_to_print(),params="outCverbose=False")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 1:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC(os.path.join(outdir,"B_from_A_2x1U.h"),set_BU_to_print(),params="outCverbose=False")

for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 2:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_ddnD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC(os.path.join(outdir,"B_from_A_2x2D.h"),set_BU_to_print(),params="outCverbose=False")
for i in range(DIM):
    BU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            if j is 2:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dupD[k][j]
            else:
                BU[i] += LeviCivitaUUU[i][j][k] * AD_dD[k][j]

fin.FD_outputC(os.path.join(outdir,"B_from_A_2x2U.h"),set_BU_to_print(),params="outCverbose=False")


Wrote to file "GiRaFFE_HO/B_from_A_2x0D.h"
Wrote to file "GiRaFFE_HO/B_from_A_2x0U.h"
Wrote to file "GiRaFFE_HO/B_from_A_2x1D.h"
Wrote to file "GiRaFFE_HO/B_from_A_2x1U.h"
Wrote to file "GiRaFFE_HO/B_from_A_2x2D.h"
Wrote to file "GiRaFFE_HO/B_from_A_2x2U.h"


<a id='ccode'></a>

## Step 2: C code \[Back to [top](#toc)\]
$$\label{ccode}$$

With the header files created to actually calculate $B^i$ from $A_k$, we will write the C code to direct how those are used. We will start by including needed header files. We will then define `REAL` as type `double` (so that we can easily change it everywhere if we want to use a different precision) and the `IDX` macros that we use to access specific points in the gridfunction arrays. 

In [6]:
%%writefile $outdir/driver_AtoB.c
#include <math.h>
#include <stdio.h>
#include <stdlib.h>

#ifndef REAL
#define REAL double
#include "NGHOSTS.h" // A NRPy+-generated file, which is set based on FD_CENTDERIVS_ORDER.
#include "../CurviBoundaryConditions/gridfunction_defines.h"
#define IDX4(g,i,j,k) \
( (i) + Nxx_plus_2NGHOSTS[0] * ( (j) + Nxx_plus_2NGHOSTS[1] * ( (k) + Nxx_plus_2NGHOSTS[2] * (g) ) ) )
#define IDX3(i,j,k) ( (i) + Nxx_plus_2NGHOSTS[0] * ( (j) + Nxx_plus_2NGHOSTS[1] * (k) ) )
// Assuming idx = IDX3(i,j,k). Much faster if idx can be reused over and over:
#define IDX4pt(g,idx)   ( (idx) + (Nxx_plus_2NGHOSTS[0]*Nxx_plus_2NGHOSTS[1]*Nxx_plus_2NGHOSTS[2]) * (g) )
#endif

// In the standalone, these are already defined by the boundary conditions module.
/*const int MAXFACE = -1;
*const int NUL     = +0;
*const int MINFACE = +1;
*/


Overwriting GiRaFFE_HO//driver_AtoB.c


<a id='a2b'></a>

### Step 2.a: Deciding which header file to use \[Back to [top](#toc)\]
$$\label{a2b}$$

The rest of this file will consist of two functions. The first, `AtoB`, will loop over the region specified by `i0min`, `i0max`, `i1min`, `i1max`, `i2min`, and `i2max`. At each point therein, it will calculate $B^i$ at the specified order. Note that if `ORDER = 0` is passed to this function, it will instead use second-order derivatives along with up- or downwinded derivatives as appropriate, which is specified by the parameters `FACEX0`, `FACEX1`, and `FACEX2`.

In [7]:
%%writefile -a $outdir/driver_AtoB.c
// Declare boundary condition FACE_UPDATE function,
// which fills in the ghost zones with successively
// lower order finite differencing
void AtoB(const int ORDER, const int Nxx_plus_2NGHOSTS[3], const REAL *in_gfs, REAL *aux_gfs, const REAL dxx[3],
          const int i0min, const int i0max, 
          const int i1min, const int i1max, 
          const int i2min, const int i2max, 
          const int FACEX0, const int FACEX1, const int FACEX2) {
  
  const REAL invdx0 = 1.0 / dxx[0];
  const REAL invdx1 = 1.0 / dxx[1];
  const REAL invdx2 = 1.0 / dxx[2];

  if(ORDER==8) {
    //printf("Computing A to B with Order = 8...\n");
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_8.h"
    }
  } else if(ORDER==6) {
    //printf("Computing A to B with Order = 6...\n");
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_6.h"
    }
  } else if(ORDER==4) {
    //printf("Computing A to B with Order = 4...\n");
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_4.h"
    }
  } else if(ORDER==2) {
    //printf("Computing A to B with Order = 2...\n");
    for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
        #include "B_from_A_2.h"
    } 
  } else if(ORDER==0) {
    if(FACEX0==MAXFACE) {
    //printf("Computing A to B at x = max...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x0D.h"
        }
    } else if(FACEX0==MINFACE) {
    //printf("Computing A to B at x = min...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x0U.h"
        }
    } else if(FACEX1==MAXFACE) {
    //printf("Computing A to B at y = max...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x1D.h"
        }
    } else if(FACEX1==MINFACE) {
    //printf("Computing A to B at y = min...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x1U.h"
        }
    } else if(FACEX2==MAXFACE) {
    //printf("Computing A to B at z = max...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x2D.h"
        }
    } else if(FACEX2==MINFACE) {
    //printf("Computing A to B at z = min...\n");
        for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
            #include "B_from_A_2x2U.h"
        }
    } else {
        printf("ERROR. FACEX parameters not set properly.\n");
        exit(1);
    }
  } else {
    printf("ERROR. ORDER = %d not supported!\n",ORDER);
    exit(1);
  }
}


Appending to GiRaFFE_HO//driver_AtoB.c


<a id='driver'></a>

### Step 2.b: Looping over all ghostzones \[Back to [top](#toc)\]
$$\label{driver}$$

This function is responsible for driving the logic needed to compute $B^i$ from $A_i$ whenever it is needed in the main code (typically done along with applying BCs after a timestep in the main code). We first calculate the order that we want on the interior of the grid from the parameter `NGHOSTS`. We set the boundaries `imin` and `imax` such that they define the interior of the grid. We also poison the values of $B^i$ by setting them to `NaN` everywhere: this will allow us to immediately tell if something is causing us to skip a point in the rest of the algorithm. 

With the preliminaries handled, we call `AtoB` at `ORDER` (which is being used for the rest of the derivatives throughout `GiRaFFE`). Note that `imin` and `imax` are passed such that they define the interior of the grid, and that the `FACEX` parameters are all `NUL` here. 

In [8]:
%%writefile -a $outdir/driver_AtoB.c
void driver_A_to_B(const int Nxx[3],const int Nxx_plus_2NGHOSTS[3], const REAL dxx[3],const REAL *in_gfs,REAL *aux_gfs) {
  
  int ORDER = NGHOSTS*2;
  for(int ii=0;ii<Nxx_plus_2NGHOSTS[2]*Nxx_plus_2NGHOSTS[1]*Nxx_plus_2NGHOSTS[0];ii++) {
      aux_gfs[IDX4pt(BU0GF,ii)] = 1.0 / 0.0;
      aux_gfs[IDX4pt(BU1GF,ii)] = 1.0 / 0.0;
      aux_gfs[IDX4pt(BU2GF,ii)] = 1.0 / 0.0;
  }

  //printf("Starting A to B driver with Order = %d...\n",ORDER);
  int imin[3] = { NGHOSTS, NGHOSTS, NGHOSTS };
  int imax[3] = { Nxx[0]+NGHOSTS, Nxx[1]+NGHOSTS, Nxx[2]+NGHOSTS };
  AtoB(ORDER, Nxx_plus_2NGHOSTS, in_gfs, aux_gfs,dxx, imin[0],imax[0],imin[1],imax[1],imin[2],imax[2], NUL,NUL,NUL);


Appending to GiRaFFE_HO//driver_AtoB.c


Now, we will handle the ghost zones. We first step down the order the next lower even number. Then, for each face (taking care to match the region specified bythe `imin` and `imax` parameters with the appropriate `FACEX` values), we calculate $B^i$ in the innermost ghostzone. We also decrement `imin` and increment `imax`. This serves two purposes: the next time through the loop, `AtoB` will operate on the next-outer ghost zone, and the size of each face will increase, eventually allowing us to act on each point. 

We then repeat the process for each lower even order until the reach the outermost ghost zone. 

In [9]:
%%writefile -a $outdir/driver_AtoB.c
  while(ORDER>0) {
      // After updating each face, adjust imin[] and imax[] 
      //   to reflect the newly-updated face extents.
      ORDER -= 2;
      AtoB(ORDER, Nxx_plus_2NGHOSTS, in_gfs, aux_gfs,dxx, imin[0]-1,imin[0], imin[1],imax[1], imin[2],imax[2], MINFACE,NUL,NUL); 
      if(ORDER!=0) imin[0]--;
      AtoB(ORDER, Nxx_plus_2NGHOSTS, in_gfs, aux_gfs,dxx, imax[0],imax[0]+1, imin[1],imax[1], imin[2],imax[2], MAXFACE,NUL,NUL); 
      if(ORDER!=0) imax[0]++;

      AtoB(ORDER, Nxx_plus_2NGHOSTS, in_gfs, aux_gfs,dxx, imin[0],imax[0], imin[1]-1,imin[1], imin[2],imax[2], NUL,MINFACE,NUL); 
      if(ORDER!=0) imin[1]--;
      AtoB(ORDER, Nxx_plus_2NGHOSTS, in_gfs, aux_gfs,dxx, imin[0],imax[0], imax[1],imax[1]+1, imin[2],imax[2], NUL,MAXFACE,NUL); 
      if(ORDER!=0) imax[1]++;

      AtoB(ORDER, Nxx_plus_2NGHOSTS, in_gfs, aux_gfs,dxx, imin[0],imax[0], imin[1],imax[1], imin[2]-1,imin[2], NUL,NUL,MINFACE); 
      if(ORDER!=0) imin[2]--;
      AtoB(ORDER, Nxx_plus_2NGHOSTS, in_gfs, aux_gfs,dxx, imin[0],imax[0], imin[1],imax[1], imax[2],imax[2]+1, NUL,NUL,MAXFACE); 
      if(ORDER!=0) imax[2]++;
    }
}


Appending to GiRaFFE_HO//driver_AtoB.c


In [10]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-GiRaFFE_HO_C_code_library-A2B.ipynb
!pdflatex -interaction=batchmode Tutorial-GiRaFFE_HO_C_code_library-A2B.tex
!pdflatex -interaction=batchmode Tutorial-GiRaFFE_HO_C_code_library-A2B.tex
!pdflatex -interaction=batchmode Tutorial-GiRaFFE_HO_C_code_library-A2B.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-GiRaFFE_HO_C_code_library-A2B.ipynb to latex
[NbConvertApp] Writing 53267 bytes to Tutorial-GiRaFFE_HO_C_code_library-A2B.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
